In [2]:
open System
open System.IO

In [3]:
let testInput=""".....
.S-7.
.|.|.
.L-J.
....."""

In [4]:
let mapFromInput (str:String) =
    let temp = 
        str.Trim().Split('\n')
        |> Array.map (fun x -> Array.ofSeq (x.Trim()))
    Array2D.init temp.Length (temp[0].Length) (fun row col -> temp[row][col])
let testMap = mapFromInput testInput

In [5]:
type Direction = 
    | Up
    | Down
    | Left
    | Right
type Position = {Row:int; Col:int; Looking: Direction}
let nextPosition (pos:Position) (d:Direction) : Position = 
    match d with 
    | Up -> {pos with Row=pos.Row - 1}
    | Down -> {pos with Row = pos.Row + 1}
    | Left -> {pos with Col = pos.Col - 1}
    | Right -> {pos with Col = pos.Col + 1}
let nextDirection (pipe:Char) (d:Direction) = 
    match d, pipe with 
    | Right, 'J' -> Up
    | Right, '7' -> Down
    | Down, 'J' -> Left 
    | Down, 'L' -> Right
    | Left, 'L' -> Up
    | Left, 'F' -> Down
    | Up, 'F' -> Right
    | Up, '7' -> Left
    | _ -> d // These are the cases where we go into a straight pipe



In [6]:
let stepInto (pipe:Char) (pos:Position) = 
    let newDirection = nextDirection pipe pos.Looking 
    {nextPosition pos pos.Looking with Looking = newDirection}

In [7]:
let startingPos = {Row=0;Col=0; Looking=Down}
startingPos
|> stepInto '|' 
|> stepInto 'L'
|> stepInto '-'
|> stepInto 'J'
|> stepInto '|'
|> stepInto '7'
|> stepInto '-'
|> stepInto 'F'



{ Row = 0\n Col = 0\n Looking = Down } Row 0 Col 0 Looking Down

In [8]:
let getPipe (map: Char[,]) (pos:Position) = 
    map[pos.Row,pos.Col]

In [9]:
let prettyPrint boardArray =
    for r = 0 to Array2D.length1 boardArray - 1 do
        printf "\n"
        for c = 0 to Array2D.length2 boardArray - 1 do
            printf "%A " boardArray.[r, c]

In [10]:
let findIndex predicate (array : _[,]) = 
    let len1 = Array2D.length1 array 
    let len2 = Array2D.length2 array 

    let rec go i j  =
        if j>=len2 then 
            failwith "Index not found" 
        elif i>=len1 then 
            go 0 (j + 1) 
        elif predicate array[i,j] then (i,j)
        else 
            go (i + 1) j

    go 0 0

let getDistances (map: Char[,]) (startDir: Direction): int[,] = 
    let mutable distances : int[,] = Array2D.zeroCreate (Array2D.length1 map) (Array2D.length2 map)
    let mutable dis = 0
    let r,c = findIndex (fun x -> x = 'S') map 
    let mutable pos = {Row=r;Col=c;Looking=startDir}

    let mutable pipe:Char = '.' 
    while pipe <> 'S' do 
        pipe <- nextPosition pos pos.Looking |> getPipe map 
        pos <- pos |> stepInto pipe
        dis <- dis + 1
        distances[pos.Row, pos.Col] <- dis

    distances
    |> Array2D.map (fun x -> min x (dis - x))
    

In [11]:
getDistances testMap Down 
|> Seq.cast<int> 
|> Seq.max

4

In [12]:
let input = File.ReadAllText("input_10.txt")
let map = mapFromInput input

In [13]:
getDistances map Right
|> Seq.cast<int>
|> Seq.max

6831

# Part 2

In [14]:
// Get coordinates of the main loop
let getLoopMask (map: Char[,]) (startDir: Direction): bool[,] = 
    let mutable mask : bool[,] = Array2D.create (Array2D.length1 map) (Array2D.length2 map) false
    let r,c = findIndex (fun x -> x = 'S') map 
    let mutable pos = {Row=r;Col=c;Looking=startDir}

    let mutable pipe:Char = '.' 
    while pipe <> 'S' do 
        pipe <- nextPosition pos pos.Looking |> getPipe map 
        pos <- pos |> stepInto pipe
        mask[pos.Row, pos.Col] <- true

    mask

let jordanPoint (pipe:Char) = 

    match pipe with 
    // | 'F' | '7' | 'L' | 'J' -> 0.5
    | '-' -> 1.0
    | _ -> 0.0

// let isInLoop (map: Char[,]) (mask:bool[,]) row col  = 
//     if mask[row,col] then 
//         false
//     else 
//         let mutable crossings = 0.
//         let mutable currentRow = row - 1
//         while currentRow >= 0 do 
//             if mask[currentRow, col] then 
//                 crossings <- crossings + (jordanPoint map[currentRow, col] )
//             currentRow <- currentRow - 1
//         int(crossings) % 2 = 1

In [15]:
let isInLoop (map: Char[,]) (mask:bool[,]) row col  = 
    if mask[row,col] then 
        false
    else 
        let mutable crossings = 0
        let mutable newPoints = 0
        let mutable currentRow = row - 1
        let mutable lastCorner = None
        let mutable currentPipe = '.'
        while currentRow >= 0 do 
            if mask[currentRow, col] then 
                currentPipe <- map[currentRow, col]
                newPoints <- 
                    match lastCorner, currentPipe with 
                    | Some 'L', '7' -> lastCorner <- None; 1 
                    | Some 'J', 'F' -> lastCorner <- None; 1
                    | _, '-' -> 1 
                    | _ , 'L' | _, 'J' -> lastCorner <- Some currentPipe; 0 
                    | _ -> 0 
                crossings <- crossings + newPoints
            currentRow <- currentRow - 1
        crossings % 2 = 1

In [16]:
let solve2 (sReplacement:Char) (startDir: Direction) (map:Char[,]) = 
    let mask = getLoopMask map startDir
    let r,c = findIndex (fun x -> x = 'S') map 
    let localMap = Array2D.copy map
    localMap[r,c] <- sReplacement
    let isInThisLoop = isInLoop localMap mask 
    Array2D.init
        (Array2D.length1 map)
        (Array2D.length2 map)
        isInThisLoop
    // |> Seq.cast<bool>
    // |> Seq.sumBy (fun x -> if x then 1 else 0)

In [27]:
solve2 '-' Right map
|> Seq.cast<bool>
|> Seq.sumBy (fun x -> if x then 1 else 0)

305

In [18]:
let testInput2 = """FF7FSF7F7F7F7F7F---7
L|LJ||||||||||||F--J
FL-7LJLJ||||||LJL-77
F--JF--7||LJLJIF7FJ-
L---JF-JLJIIIIFJLJJ7
|F|F-JF---7IIIL7L|7|
|FFJF7L7F-JF7IIL---7
7-L-JL7||F7|L7F-7F7|
L.L7LFJ|||||FJL7||LJ
L7JLJL-JLJLJL--JLJ.L"""

let testMap2 = mapFromInput testInput2
let testMask2 = getLoopMask testMap2 Down

In [19]:
testMask2
|> Array2D.mapi (fun r c x -> if x then testMap2[r,c] else ' ')
|> prettyPrint


' ' 'F' '7' 'F' 'S' 'F' '7' 'F' '7' 'F' '7' 'F' '7' 'F' '7' 'F' '-' '-' '-' '7' 
' ' '|' 'L' 'J' '|' '|' '|' '|' '|' '|' '|' '|' '|' '|' '|' '|' 'F' '-' '-' 'J' 
' ' 'L' '-' '7' 'L' 'J' 'L' 'J' '|' '|' '|' '|' '|' '|' 'L' 'J' 'L' '-' '7' ' ' 
'F' '-' '-' 'J' 'F' '-' '-' '7' '|' '|' 'L' 'J' 'L' 'J' ' ' 'F' '7' 'F' 'J' ' ' 
'L' '-' '-' '-' 'J' 'F' '-' 'J' 'L' 'J' ' ' ' ' ' ' ' ' 'F' 'J' 'L' 'J' ' ' ' ' 
' ' ' ' ' ' 'F' '-' 'J' 'F' '-' '-' '-' '7' ' ' ' ' ' ' 'L' '7' ' ' ' ' ' ' ' ' 
' ' ' ' 'F' 'J' 'F' '7' 'L' '7' 'F' '-' 'J' 'F' '7' ' ' ' ' 'L' '-' '-' '-' '7' 
' ' ' ' 'L' '-' 'J' 'L' '7' '|' '|' 'F' '7' '|' 'L' '7' 'F' '-' '7' 'F' '7' '|' 
' ' ' ' ' ' ' ' ' ' 'F' 'J' '|' '|' '|' '|' '|' 'F' 'J' 'L' '7' '|' '|' 'L' 'J' 
' ' ' ' ' ' ' ' ' ' 'L' '-' 'J' 'L' 'J' 'L' 'J' 'L' '-' '-' 'J' 'L' 'J' ' ' ' ' 

In [20]:
solve2 '7' Down testMap2
|> Array2D.map (fun x -> if x then 'I' else 'O')
|> prettyPrint


'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 
'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 
'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 
'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'I' 'O' 'O' 'O' 'O' 'O' 
'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'I' 'I' 'I' 'I' 'O' 'O' 'O' 'O' 'O' 'O' 
'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'I' 'I' 'I' 'O' 'O' 'O' 'O' 'O' 'O' 
'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'I' 'I' 'O' 'O' 'O' 'O' 'O' 
'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 
'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 
'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 

In [21]:
solve2 '7' Down testMap2
|> Seq.cast<bool> 
|> Seq.sumBy (fun x -> if x then 1 else 0)

10

In [23]:
let testInput3 = """..........
.S------7.
.|F----7|.
.||OOOO||.
.||OOOO||.
.|L-7F-J|.
.|II||II|.
.L--JL--J.
.........."""
let testMap3 = mapFromInput testInput3
let testMask3 = getLoopMask testMap3 Down

In [26]:
testMask3
|> Array2D.mapi (fun r c x -> if x then testMap2[r,c] else ' ')
|> prettyPrint


' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' 
' ' '|' 'L' 'J' '|' '|' '|' '|' '|' ' ' 
' ' 'L' '-' '7' 'L' 'J' 'L' 'J' '|' ' ' 
' ' '-' '-' ' ' ' ' ' ' ' ' '7' '|' ' ' 
' ' '-' '-' ' ' ' ' ' ' ' ' 'J' 'L' ' ' 
' ' 'F' '|' 'F' '-' 'J' 'F' '-' '-' ' ' 
' ' 'F' ' ' ' ' 'F' '7' ' ' ' ' 'F' ' ' 
' ' '-' 'L' '-' 'J' 'L' '7' '|' '|' ' ' 
' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' 

In [25]:
solve2 'F' Down testMap3
|> Array2D.map (fun x -> if x then 'I' else 'O')
|> prettyPrint


'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 
'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 
'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 
'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 
'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 
'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 
'O' 'O' 'I' 'I' 'O' 'O' 'I' 'I' 'O' 'O' 
'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 
'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 